### Ontario Gateway: 1 year

In [68]:
import numpy as np
import pandas as pd

In [69]:
# Aircraft info: Boeing 757-200, Airbus A340-200, Airbus A340-300, respectively
num_aircraft = np.array([47, 15, 24])
num_flights_per_day = np.array([6, 2.25, 2])
replacement_cost = np.array([56400000, 78900000, 88500000], dtype=np.int64)
fleet_value = np.dot(num_aircraft, replacement_cost)

num_days_per_year = 342
accident_rate = 1 / 5000000

num_flights_per_year = np.ceil(num_aircraft * num_flights_per_day * num_days_per_year).astype(int)

In [70]:
N = 10000000
M = len(num_aircraft)

dat_accident = np.zeros((N, M))

for k in range(M):
    dat_accident[:, k] = np.random.binomial(num_flights_per_year[k], accident_rate, N)

aircraft_losses = np.sum(dat_accident * replacement_cost, axis=1)
incidental_losses = np.random.uniform(1000000, 5000000, N)
total_losses = aircraft_losses + incidental_losses

In [71]:
cost_RCNC1 = 0.0045 * fleet_value + 0.10 * total_losses
cost_RCNC2 = 0.001 * fleet_value + np.minimum(0.90 * total_losses, 0.01 * fleet_value)
cost_CTC = 13000000 + 0.10 * np.minimum(total_losses, 80000000) + np.maximum(total_losses - 80000000, 0)
cost_HIC = 0.00165 * fleet_value + np.minimum(total_losses, 24000000)
cost_SELF = total_losses

In [72]:
exceed_37M_RCNC1 = np.mean(cost_RCNC1 > 37000000)
exceed_37M_RCNC2 = np.mean(cost_RCNC2 > 37000000)
exceed_37M_CTC = np.mean(cost_CTC > 37000000)
exceed_37M_HIC = np.mean(cost_HIC > 37000000)
exceed_37M_SELF = np.mean(cost_SELF > 37000000)

In [73]:
policy_summary = pd.DataFrame({
    "policy": ["RCNC1", "RCNC2", "CTC", "HIC", "SELF"],
    "exp_cost": [np.mean(cost_RCNC1), np.mean(cost_RCNC2), np.mean(cost_CTC), np.mean(cost_HIC), np.mean(cost_SELF)],
    "sd_cost": [np.std(cost_RCNC1, ddof = 1), np.std(cost_RCNC2, ddof = 1), np.std(cost_CTC, ddof = 1), np.std(cost_HIC, ddof = 1), np.std(cost_SELF, ddof = 1)],
    "exceed37M": [exceed_37M_RCNC1, exceed_37M_RCNC2, exceed_37M_CTC, exceed_37M_HIC, exceed_37M_SELF]
})
policy_summary['exp_cost'] = policy_summary['exp_cost'].map(lambda x: f"${x:,.0f}")
policy_summary['sd_cost'] = policy_summary['sd_cost'].map(lambda x: f"${x:,.0f}")
policy_summary['exceed37M'] = policy_summary['exceed37M'].map(lambda x: f"{x*100:,.2f}%")
print(policy_summary)

  policy     exp_cost      sd_cost exceed37M
0  RCNC1  $27,267,914   $1,012,259     0.03%
1  RCNC2   $9,936,827   $8,144,181     2.45%
2    CTC  $13,505,997   $1,744,971     0.03%
3    HIC  $13,345,713   $3,440,533     0.00%
4   SELF   $4,555,640  $10,122,590     2.45%
